In [1]:
import pandas as pd
import numpy as np
import os
import re
import math

In [2]:
#df = pd.read_csv("ADNI-FULL-PostProc_11_29_2022.csv")
df = pd.read_csv("ADNI-FULL-PostProc_3_01_2023.csv")
df

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,I1557049,941_S_7041,MCI,F,73,bl,PET,Co-registered Dynamic,Processed,3/09/2022,DCM,11/29/2022
1,I1557046,941_S_7041,MCI,F,73,bl,PET,"Coreg, Avg, Standardized Image and Voxel Size",Processed,3/09/2022,DCM,12/02/2022
2,I1557048,941_S_7041,MCI,F,73,bl,PET,"Co-registered, Averaged",Processed,3/09/2022,DCM,12/02/2022
3,I1614530,941_S_7041,MCI,F,73,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform 6mm Res",Processed,3/09/2022,DCM,11/29/2022
4,I1557047,941_S_7041,MCI,F,73,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,3/09/2022,DCM,11/29/2022
...,...,...,...,...,...,...,...,...,...,...,...,...
17176,I240517,002_S_0295,CN,M,90,v06,PET,"Coreg, Avg, Standardized Image and Voxel Size",Processed,6/09/2011,DCM,1/09/2023
17177,I240524,002_S_0295,CN,M,90,v06,PET,"Co-registered, Averaged",Processed,6/09/2011,DCM,1/09/2023
17178,I1592036,002_S_0295,CN,M,90,v06,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform 6mm Res",Processed,6/09/2011,DCM,12/02/2022
17179,I240519,002_S_0295,CN,M,90,v06,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,6/09/2011,DCM,12/02/2022


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17181 entries, 0 to 17180
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Image Data ID  17181 non-null  object
 1   Subject        17181 non-null  object
 2   Group          17181 non-null  object
 3   Sex            17181 non-null  object
 4   Age            17181 non-null  int64 
 5   Visit          17181 non-null  object
 6   Modality       17181 non-null  object
 7   Description    17181 non-null  object
 8   Type           17181 non-null  object
 9   Acq Date       17181 non-null  object
 10  Format         17181 non-null  object
 11  Downloaded     17181 non-null  object
dtypes: int64(1), object(11)
memory usage: 1.6+ MB


In [4]:
# Cantidad de sujetos
df['Subject'].unique().shape

(1691,)

In [5]:
# Tipos de descripcion
df['Description'].unique()

array(['Co-registered Dynamic',
       'Coreg, Avg, Standardized Image and Voxel Size',
       'Co-registered, Averaged',
       'Coreg, Avg, Std Img and Vox Siz, Uniform 6mm Res',
       'Coreg, Avg, Std Img and Vox Siz, Uniform Resolution'],
      dtype=object)

In [6]:
df[df['Description'].str.contains('Coreg, Avg, Std Img and Vox Siz, Uniform Resolution')].shape

(3762, 12)

In [7]:
df[df['Description'].str.contains('Coreg, Avg, Std Img and Vox Siz, Uniform 6mm Res')].shape

(2703, 12)

In [8]:
df[df['Description'].str.contains('Coreg, Avg, Std Img and Vox Siz, Uniform 6mm Res')]['Subject'].unique().shape

(1400,)

In [9]:
df[df['Description'].str.contains('Coreg, Avg, Std Img and Vox Siz, Uniform Resolution')]['Subject'].unique().shape

(1687,)

Varones únicos:

In [10]:
df[df['Description'].str.contains('Coreg, Avg, Std Img and Vox Siz, Uniform Resolution') & (df['Sex'] == 'M')]['Subject'].unique().shape

(937,)

Mujeres únicas:

In [11]:
df[df['Description'].str.contains('Coreg, Avg, Std Img and Vox Siz, Uniform Resolution') & (df['Sex'] == 'F')]['Subject'].unique().shape

(750,)

In [12]:
dfu = df[df['Description'].str.contains('Coreg, Avg, Std Img and Vox Siz, Uniform Resolution')]
dfu.shape

(3762, 12)

Grupos:

In [21]:
dfu['Group'].unique()

array(['MCI', 'AD', 'SMC', 'EMCI', 'LMCI', 'CN'], dtype=object)

In [22]:
dfu.groupby('Group').describe()

Age                                                   
        count       mean       std   min   25%   50%   75%   max
Group                                                           
AD      555.0  75.888288  7.673043  55.0  71.0  76.0  81.5  92.0
CN      949.0  76.988409  5.945682  56.0  73.0  77.0  81.0  96.0
EMCI    534.0  72.440075  7.566786  55.0  67.0  72.0  78.0  90.0
LMCI    249.0  73.349398  7.733099  55.0  68.0  74.0  78.0  94.0
MCI    1364.0  76.071848  7.587347  55.0  71.0  76.0  82.0  96.0
SMC     111.0  72.774775  5.704360  60.0  68.0  73.0  77.0  90.0

In [31]:
dfug = dfu.groupby(by=['Group', 'Sex'])
dfug.describe()

Age                                                     
           count       mean       std   min    25%   50%    75%   max
Group Sex                                                            
AD    F    225.0  75.106667  7.704776  55.0  71.00  75.0  80.00  92.0
      M    330.0  76.421212  7.617060  56.0  72.00  77.0  82.00  92.0
CN    F    405.0  76.580247  6.299191  56.0  72.00  77.0  81.00  96.0
      M    544.0  77.292279  5.654986  60.0  73.00  77.0  81.00  91.0
EMCI  F    233.0  71.759657  8.189707  56.0  65.00  71.0  78.00  90.0
      M    301.0  72.966777  7.015618  55.0  68.00  73.0  77.00  90.0
LMCI  F    116.0  71.801724  7.842160  55.0  66.00  73.0  77.25  91.0
      M    133.0  74.699248  7.404938  56.0  70.00  75.0  80.00  94.0
MCI   F    493.0  74.902637  8.015651  55.0  70.00  75.0  81.00  96.0
      M    871.0  76.733639  7.255454  57.0  72.00  77.0  82.00  94.0
SMC   F     64.0  72.390625  5.467413  60.0  67.75  73.0  76.25  83.0
      M     47.0  73.297872  6.032214  65.0  68.00  72.0  77.50  90.0

In [32]:
dfu[dfu['Sex'] == 'M'].shape

(2226, 12)

In [33]:
dfu[dfu['Sex'] == 'F'].shape

(1536, 12)

In [34]:
dfu['Age'].describe()

count    3762.000000
mean       75.482988
std         7.349110
min        55.000000
25%        71.000000
50%        76.000000
75%        81.000000
max        96.000000
Name: Age, dtype: float64

In [37]:
dfu.groupby(by='Sex')['Age'].describe()

,count,mean,std,min,25%,50%,75%,max
Sex,,,,,,,,
F,1536.0,74.559245,7.667547,55.0,70.0,75.0,80.0,96.0
M,2226.0,76.120395,7.052603,55.0,72.0,76.0,81.0,94.0


In [35]:
dfug['Age'].describe()

count       mean       std   min    25%   50%    75%   max
Group Sex                                                            
AD    F    225.0  75.106667  7.704776  55.0  71.00  75.0  80.00  92.0
      M    330.0  76.421212  7.617060  56.0  72.00  77.0  82.00  92.0
CN    F    405.0  76.580247  6.299191  56.0  72.00  77.0  81.00  96.0
      M    544.0  77.292279  5.654986  60.0  73.00  77.0  81.00  91.0
EMCI  F    233.0  71.759657  8.189707  56.0  65.00  71.0  78.00  90.0
      M    301.0  72.966777  7.015618  55.0  68.00  73.0  77.00  90.0
LMCI  F    116.0  71.801724  7.842160  55.0  66.00  73.0  77.25  91.0
      M    133.0  74.699248  7.404938  56.0  70.00  75.0  80.00  94.0
MCI   F    493.0  74.902637  8.015651  55.0  70.00  75.0  81.00  96.0
      M    871.0  76.733639  7.255454  57.0  72.00  77.0  82.00  94.0
SMC   F     64.0  72.390625  5.467413  60.0  67.75  73.0  76.25  83.0
      M     47.0  73.297872  6.032214  65.0  68.00  72.0  77.50  90.0

In [29]:
dfu['Subject'].unique().shape

(1687,)

In [43]:
dfu.groupby(by=['Sex'])['Age'].describe()

,count,mean,std,min,25%,50%,75%,max
Sex,,,,,,,,
F,1536.0,74.559245,7.667547,55.0,70.0,75.0,80.0,96.0
M,2226.0,76.120395,7.052603,55.0,72.0,76.0,81.0,94.0


In [44]:
df.describe()

,Age
count,17181.000000
mean,75.386532
std,7.374799
min,55.000000
25%,71.000000
50%,76.000000
75%,81.000000
max,96.000000


In [43]:
dfu6 = df[df['Description'].str.contains('Coreg, Avg, Std Img and Vox Siz, Uniform 6mm Res')]
dfu6.shape

(2703, 12)

In [44]:
dfug6 = dfu6.groupby(by=['Group', 'Sex'])
dfug6.describe()

Age                                                    
           count       mean       std   min   25%   50%    75%   max
Group Sex                                                           
AD    F    158.0  74.101266  8.018442  55.0  71.0  74.0  79.00  92.0
      M    225.0  75.680000  7.522276  56.0  71.0  76.0  81.00  92.0
CN    F    302.0  76.178808  6.687705  56.0  72.0  76.0  81.00  96.0
      M    375.0  77.248000  5.890248  60.0  73.0  77.0  81.00  91.0
EMCI  F    199.0  71.628141  8.336509  56.0  65.0  70.0  78.00  90.0
      M    255.0  73.050980  7.044945  55.0  68.0  73.0  77.50  90.0
LMCI  F    103.0  72.009709  8.057995  55.0  66.0  73.0  78.00  91.0
      M    116.0  74.681034  7.449072  56.0  70.0  75.0  80.00  94.0
MCI   F    354.0  75.121469  8.190331  56.0  70.0  75.0  81.00  96.0
      M    519.0  76.718690  7.441526  57.0  72.0  77.0  82.00  94.0
SMC   F     57.0  72.894737  5.350848  60.0  68.0  73.0  77.00  83.0
      M     40.0  73.525000  6.308267  65.0  68.0  72.5  78.25  90.0

In [45]:
dfu6.groupby(by=['Group']).describe()

Age                                                   
       count       mean       std   min   25%   50%   75%   max
Group                                                          
AD     383.0  75.028721  7.759589  55.0  71.0  75.0  81.0  92.0
CN     677.0  76.771049  6.276366  56.0  73.0  77.0  81.0  96.0
EMCI   454.0  72.427313  7.661881  55.0  67.0  72.0  78.0  90.0
LMCI   219.0  73.424658  7.838246  55.0  68.0  74.0  79.0  94.0
MCI    873.0  76.071019  7.788900  56.0  71.0  76.0  82.0  96.0
SMC     97.0  73.154639  5.741552  60.0  68.0  73.0  77.0  90.0

In [46]:
dfu6.groupby(by=['Sex']).describe()

Age                                                   
      count       mean       std   min   25%   50%   75%   max
Sex                                                           
F    1173.0  74.282182  7.870951  55.0  69.0  74.0  80.0  96.0
M    1530.0  75.846405  7.159717  55.0  71.0  76.0  81.0  94.0

In [47]:
dfu6.groupby(by=['Sex'])['Age'].describe()

,count,mean,std,min,25%,50%,75%,max
Sex,,,,,,,,
F,1173.0,74.282182,7.870951,55.0,69.0,74.0,80.0,96.0
M,1530.0,75.846405,7.159717,55.0,71.0,76.0,81.0,94.0


In [48]:
dfu6['Age'].mean()

75.16759156492786

In [52]:
dfu6['Age'].std()

7.515369916504292

Varones únicos:

In [53]:
df[df['Description'].str.contains('Coreg, Avg, Std Img and Vox Siz, Uniform 6mm Res') & (df['Sex'] == 'M')]['Subject'].unique().shape

(762,)

Mujeres únicas:

In [54]:
df[df['Description'].str.contains('Coreg, Avg, Std Img and Vox Siz, Uniform 6mm Res') & (df['Sex'] == 'F')]['Subject'].unique().shape

(638,)

# PiB-PET

In [61]:
dfp = pd.read_csv("ADNI1-PIB-PET_12_05_2022.csv")
dfp

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,I174162,128_S_0266,AD,M,87,4,PET,ADNI Brain PET: Raw PIB,Original,5/17/2007,HRRT,NaN
1,I179432,128_S_1407,MCI,F,76,4,PET,ADNI Brain PET: Raw PIB,Original,7/29/2008,HRRT,NaN
2,I189950,128_S_0500,CN,M,79,4,PET,ADNI Brain PET: Raw PIB,Original,5/29/2007,HRRT,NaN
3,I189456,128_S_0135,MCI,F,84,6,PET,ADNI Brain PET: Raw PIB,Original,2/22/2008,HRRT,NaN
4,I178700,128_S_1430,AD,F,84,2,PET,ADNI Brain PET: Raw PIB,Original,12/20/2007,HRRT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
223,I95165,127_S_0112,MCI,M,73,6,PET,ADNI 11C-PIB 0112,Original,2/29/2008,DCM,NaN
224,I78230,018_S_0682,AD,M,69,4,PET,3D BRAIN ADNI PIB,Original,9/04/2007,DCM,NaN
225,I76826,037_S_0627,AD,F,60,4,PET,ADNI PET Brain PIB,Original,10/04/2007,DCM,NaN
226,I103608,007_S_0344,MCI,M,81,6,PET,TRANSAXIAL:BRAIN 3D:PIB:ADNI CTAC,Original,4/23/2008,DCM,NaN


In [63]:
dgp = dfp.groupby(by='Group')
dgp['Group'].unique()

Group
AD      [AD]
CN      [CN]
MCI    [MCI]
Name: Group, dtype: object

In [64]:
dgp.describe()

Age                                                      Visit  \
       count       mean       std   min    25%   50%   75%   max  count   
Group                                                                     
AD      34.0  74.500000  8.239778  55.0  69.25  73.5  80.5  88.0   34.0   
CN      49.0  79.285714  5.392897  71.0  74.00  80.0  84.0  90.0   49.0   
MCI    145.0  75.972414  7.959049  56.0  71.00  76.0  82.0  90.0  145.0   

                 ...            Downloaded                               
           mean  ...  75%   max      count mean std min 25% 50% 75% max  
Group            ...                                                     
AD     4.529412  ...  6.0   6.0        0.0  NaN NaN NaN NaN NaN NaN NaN  
CN     6.000000  ...  8.0  10.0        0.0  NaN NaN NaN NaN NaN NaN NaN  
MCI    5.317241  ...  6.0  10.0        0.0  NaN NaN NaN NaN NaN NaN NaN  

[3 rows x 24 columns]

Datos preprocesados

In [65]:
dfp = pd.read_csv("ADNI1-PIB-Preprocessed_12_06_2022.csv")
dfp

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,I164032,041_S_1002,CN,F,80,8,PET,"PIB Co-registered, Averaged",Processed,12/17/2009,DCM,12/05/2022
1,I135559,127_S_0112,MCI,M,74,8,PET,"PIB Co-registered, Averaged",Processed,1/23/2009,DCM,12/05/2022
2,I109459,094_S_1402,AD,M,70,4,PET,PIB Co-registered Dynamic,Processed,6/04/2008,DCM,12/05/2022
3,I1598634,041_S_1423,MCI,M,71,4,PET,"PIB Coreg, Avg, Std Img and Vox Siz, Uniform 6...",Processed,8/04/2008,DCM,12/05/2022
4,I80971,041_S_0898,CN,F,84,4,PET,"PIB Co-registered, Averaged",Processed,11/02/2007,DCM,12/05/2022
...,...,...,...,...,...,...,...,...,...,...,...,...
1033,I1598484,041_S_1002,CN,F,78,6,PET,"PIB Coreg, Avg, Std Img and Vox Siz, Uniform 6...",Processed,11/14/2008,DCM,12/05/2022
1034,I76831,041_S_0598,MCI,M,74,4,PET,PIB Co-registered Dynamic,Processed,9/26/2007,DCM,12/05/2022
1035,I190422,128_S_0500,CN,M,79,4,PET,PIB Co-registered Dynamic,Processed,5/29/2007,DCM,12/05/2022
1036,I161811,137_S_0972,CN,M,81,8,PET,"PIB Coreg, Avg, Standardized Image and Voxel Size",Processed,12/02/2009,DCM,12/05/2022


In [66]:
dfpg = dfp.groupby(by="Description")
dfpg

In [67]:
dfpg.describe()

Age             \
                                                    count       mean   
Description                                                            
PIB Co-registered Dynamic                           203.0  76.541872   
PIB Co-registered, Averaged                         203.0  76.541872   
PIB Coreg, Avg, Standardized Image and Voxel Size   228.0  76.464912   
PIB Coreg, Avg, Std Img and Vox Siz, Uniform 6m...  176.0  76.744318   
PIB Coreg, Avg, Std Img and Vox Siz, Uniform Re...  228.0  76.464912   

                                                                          \
                                                         std   min   25%   
Description                                                                
PIB Co-registered Dynamic                           7.641210  55.0  72.0   
PIB Co-registered, Averaged                         7.641210  55.0  72.0   
PIB Coreg, Avg, Standardized Image and Voxel Size   7.658665  55.0  72.0   
PIB Coreg, Avg, Std Img and Vox Siz, Uniform 6m...  7.699905  55.0  72.0   
PIB Coreg, Avg, Std Img and Vox Siz, Uniform Re...  7.658665  55.0  72.0   

                                                                      Visit  \
                                                     50%   75%   max  count   
Description                                                                   
PIB Co-registered Dynamic                           77.0  83.0  90.0  203.0   
PIB Co-registered, Averaged                         77.0  83.0  90.0  203.0   
PIB Coreg, Avg, Standardized Image and Voxel Size   77.0  83.0  90.0  228.0   
PIB Coreg, Avg, Std Img and Vox Siz, Uniform 6m...  77.0  83.0  90.0  176.0   
PIB Coreg, Avg, Std Img and Vox Siz, Uniform Re...  77.0  83.0  90.0  228.0   

                                                                             \
                                                        mean       std  min   
Description                                                                   
PIB Co-registered Dynamic                           5.295567  1.861898  2.0   
PIB Co-registered, Averaged                         5.295567  1.861898  2.0   
PIB Coreg, Avg, Standardized Image and Voxel Size   5.346491  1.872155  2.0   
PIB Coreg, Avg, Std Img and Vox Siz, Uniform 6m...  5.340909  1.881558  2.0   
PIB Coreg, Avg, Std Img and Vox Siz, Uniform Re...  5.346491  1.872155  2.0   

                                                                         
                                                    25%  50%  75%   max  
Description                                                              
PIB Co-registered Dynamic                           4.0  6.0  6.0  10.0  
PIB Co-registered, Averaged                         4.0  6.0  6.0  10.0  
PIB Coreg, Avg, Standardized Image and Voxel Size   4.0  6.0  6.0  10.0  
PIB Coreg, Avg, Std Img and Vox Siz, Uniform 6m...  4.0  6.0  6.0  10.0  
PIB Coreg, Avg, Std Img and Vox Siz, Uniform Re...  4.0  6.0  6.0  10.0

In [68]:
dfpg2 = dfp[dfp['Description'] == 'PIB Coreg, Avg, Std Img and Vox Siz, Uniform Resolution']

In [69]:
dfpg2.groupby(by=['Group', 'Sex']).describe()

Age                                                     Visit  \
          count       mean       std   min    25%   50%   75%   max count   
Group Sex                                                                   
AD    F    13.0  72.923077  9.673517  55.0  69.00  74.0  81.0  86.0  13.0   
      M    21.0  75.476190  7.298075  65.0  70.00  73.0  79.0  88.0  21.0   
CN    F    21.0  80.904762  4.194100  72.0  79.00  82.0  84.0  86.0  21.0   
      M    28.0  78.071429  5.925016  71.0  73.00  76.5  81.5  90.0  28.0   
MCI   F    48.0  74.875000  8.509382  56.0  70.75  76.0  82.0  86.0  48.0   
      M    97.0  76.515464  7.659571  60.0  72.00  77.0  83.0  90.0  97.0   

                                                         
               mean       std  min  25%  50%  75%   max  
Group Sex                                                
AD    F    4.615385  1.502135  2.0  4.0  4.0  6.0   6.0  
      M    4.476190  1.400680  2.0  4.0  4.0  6.0   6.0  
CN    F    6.095238  1.841325  4.0  4.0  6.0  8.0  10.0  
      M    5.928571  1.762334  4.0  4.0  6.0  8.0  10.0  
MCI   F    5.083333  1.843139  2.0  4.0  5.0  6.0   8.0  
      M    5.432990  1.962711  2.0  4.0  6.0  6.0  10.0

In [70]:
dfpg2.groupby(by=['Group']).describe()

Age                                                      Visit  \
       count       mean       std   min    25%   50%   75%   max  count   
Group                                                                     
AD      34.0  74.500000  8.239778  55.0  69.25  73.5  80.5  88.0   34.0   
CN      49.0  79.285714  5.392897  71.0  74.00  80.0  84.0  90.0   49.0   
MCI    145.0  75.972414  7.959049  56.0  71.00  76.0  82.0  90.0  145.0   

                                                     
           mean       std  min  25%  50%  75%   max  
Group                                                
AD     4.529412  1.419246  2.0  4.0  4.0  6.0   6.0  
CN     6.000000  1.779513  4.0  4.0  6.0  8.0  10.0  
MCI    5.317241  1.924634  2.0  4.0  6.0  6.0  10.0

In [71]:
dfpg2.groupby(by=['Sex']).describe()

Age                                                      Visit  \
     count       mean       std   min   25%   50%    75%   max  count   
Sex                                                                     
F     82.0  76.109756  8.289524  55.0  72.0  78.0  82.75  86.0   82.0   
M    146.0  76.664384  7.302600  60.0  72.0  76.0  83.00  90.0  146.0   

                                                   
         mean       std  min  25%  50%  75%   max  
Sex                                                
F    5.268293  1.846243  2.0  4.0  6.0  6.0  10.0  
M    5.390411  1.891445  2.0  4.0  6.0  6.0  10.0

In [95]:
dfpg2[dfpg2['Sex'] == 'F']['Subject'].unique().shape

(36,)

In [96]:
dfpg2[dfpg2['Sex'] == 'M']['Subject'].unique().shape

(67,)

In [72]:
dfpg3 = dfp[dfp['Description'] == 'PIB Coreg, Avg, Std Img and Vox Siz, Uniform 6mm Res']
dfpg3.groupby(by='Group').describe()

Age                                                       Visit  \
       count       mean       std   min    25%   50%    75%   max  count   
Group                                                                      
AD      26.0  75.076923  8.840466  55.0  70.25  75.0  82.75  88.0   26.0   
CN      45.0  79.377778  5.544403  71.0  74.00  80.0  84.00  90.0   45.0   
MCI    105.0  76.028571  7.984910  60.0  71.00  76.0  83.00  90.0  105.0   

                                                     
           mean       std  min  25%  50%  75%   max  
Group                                                
AD     4.461538  1.529203  2.0  4.0  4.0  6.0   6.0  
CN     6.000000  1.809068  4.0  4.0  6.0  8.0  10.0  
MCI    5.276190  1.903919  2.0  4.0  6.0  6.0  10.0

In [73]:
dfpg3.groupby(by='Sex').describe()

Age                                                      Visit  \
     count       mean       std   min    25%   50%   75%   max  count   
Sex                                                                     
F     70.0  77.314286  7.553755  55.0  73.25  79.0  83.0  86.0   70.0   
M    106.0  76.367925  7.807598  60.0  71.00  75.0  83.0  90.0  106.0   

                                                   
         mean       std  min  25%  50%  75%   max  
Sex                                                
F    5.257143  1.870331  2.0  4.0  6.0  6.0  10.0  
M    5.396226  1.895756  2.0  4.0  6.0  6.0  10.0

In [74]:
dfpg3.groupby(by=['Group', 'Sex']).describe()

Age                                                       Visit  \
          count       mean        std   min    25%   50%    75%   max count   
Group Sex                                                                     
AD    F     9.0  71.777778  10.837179  55.0  61.00  74.0  77.00  86.0   9.0   
      M    17.0  76.823529   7.350470  66.0  71.00  77.0  85.00  88.0  17.0   
CN    F    21.0  80.904762   4.194100  72.0  79.00  82.0  84.00  86.0  21.0   
      M    24.0  78.041667   6.286488  71.0  73.00  75.5  83.25  90.0  24.0   
MCI   F    40.0  76.675000   7.307837  61.0  72.75  78.0  82.25  86.0  40.0   
      M    65.0  75.630769   8.404926  60.0  70.00  75.0  83.00  90.0  65.0   

                                                         
               mean       std  min  25%  50%  75%   max  
Group Sex                                                
AD    F    4.444444  1.666667  2.0  4.0  4.0  6.0   6.0  
      M    4.470588  1.504894  2.0  4.0  4.0  6.0   6.0  
CN    F    6.095238  1.841325  4.0  4.0  6.0  8.0  10.0  
      M    5.916667  1.815792  4.0  4.0  6.0  8.0  10.0  
MCI   F    5.000000  1.811643  2.0  4.0  4.0  6.0   8.0  
      M    5.446154  1.952809  2.0  4.0  6.0  6.0  10.0

In [97]:
dfpg3[dfpg3['Sex'] == 'F']['Subject'].unique().shape

(31,)

In [98]:
dfpg3[dfpg3['Sex'] == 'M']['Subject'].unique().shape

(50,)

# Fleni

In [5]:
csv = pd.read_csv('/Users/hugom/PET-IA/Full-DBs/fleni-base-myrian/match.csv')

In [6]:
csv[csv.duplicated('pet_id', keep=False)]

Unnamed: 0  Unnamed: 0.1                                       anon_id  \
1             1             1                                           NaN   
5             5             5                                           NaN   
6             6             6  82492442-313797f9-20456fb1-e93414ad-69e5d284   
8             8             8  47e66e37-7cf35782-f172c260-1cd37119-2c6bf4db   
12           12            12                                           NaN   
..          ...           ...                                           ...   
234         234           255  50b4f5c1-43cbe80d-0d499cc4-525a4752-cb02a377   
235         235           256  f301e2e5-ef504c7a-adfe1c95-ef3b1c97-75b21599   
236         236           257  c0ed3dcd-9c7e8e50-067ce990-f4417a65-4c5b379f   
237         237           258  179ffba1-0dbc255c-ec399ad4-962ed53a-a19a0bff   
238         238           259  f1b927dc-db9ba0ef-0e204f32-8ebbc85f-be0fe49a   

                            Unnamed: 1  Participant ID               Name  \
1    no hay PET. Las RM son desde 2020          8964.0       MARIA TERESA   
5     no hay estudios de este paciente         17791.0           HORACIO^   
6                                  NaN         20864.0      ALBERTO RAMON   
8                                  NaN         29432.0        JOSE MARIA    
12                        dice anulada         47467.0   HECTOR (ANULADA)   
..                                 ...             ...                ...   
234                                NaN        703380.0          FERNANDO    
235                                NaN        712848.0      LIVIA LUCIANA   
236                                NaN        715946.0        PEDRO MARIA   
237                                NaN        717824.0           PATRICIO   
238                                NaN        718566.0  GUILLERMO AUGUSTO   

             Surname                Nombre T1        AN_MRI    Date_MRI  ...  \
1    BILBAO LA VIEJA                      NaN  9.275001e+12         NaN  ...   
5    GARCIA BELSUNCE                      NaN           NaN         NaN  ...   
6              LOPEZ                 FSPGR 3D  9.275000e+12  2015-10-21  ...   
8     SAENZ VALIENTE                      NaN           NaN         NaN  ...   
12            MASONI                      NaN           NaN         NaN  ...   
..               ...                      ...           ...         ...  ...   
234          BARROS               Ax FSPGR 3D  9.275001e+12  2020-09-11  ...   
235             DEON  Accelerated Ax IR-FSPGR  9.275001e+12  2021-01-08  ...   
236           PAILHE  Accelerated Ax IR-FSPGR  9.275001e+12  2021-02-11  ...   
237           MURPHY              Ax FSPGR 3D  9.275001e+12  2021-05-16  ...   
238          BASILIO  Accelerated Ax IR-FSPGR  9.275001e+12  2021-04-03  ...   

     gctx-bilat-occipital_rel gctx-bilat-occipital_per  \
1                         NaN                      NaN   
5                         NaN                      NaN   
6                    0.027379                     33.7   
8                    0.028084                     12.2   
12                        NaN                      NaN   
..                        ...                      ...   
234                  0.018745                      1.0   
235                  0.030842                     44.2   
236                  0.031266                     34.0   
237                  0.023596                      1.0   
238                  0.029793                     24.0   

    gctx-bilat-parietal_abs gctx-bilat-parietal_rel gctx-bilat-parietal_per  \
1                       NaN                     NaN                     NaN   
5                       NaN                     NaN                     NaN   
6              1.131856e+16                0.072109                    21.4   
8              1.138184e+15                0.070784                     1.9   
12                      NaN                     NaN                     NaN   

TODO : revisar qué está pasando con estos duplicados

In [7]:
csv

Unnamed: 0  Unnamed: 0.1                                       anon_id  \
0             0             0  601894e9-7192ca44-617903f9-b7775028-b7db3c01   
1             1             1                                           NaN   
2             2             2  26f052fa-48d0929f-43515de0-ab857928-4ee712ec   
3             3             3  9c673f58-f5bfbaf0-ff226082-410ef24e-e7d8ff4a   
4             4             4  0f5ab5b2-c7051e97-42563753-f7f14410-29b463dd   
..          ...           ...                                           ...   
234         234           255  50b4f5c1-43cbe80d-0d499cc4-525a4752-cb02a377   
235         235           256  f301e2e5-ef504c7a-adfe1c95-ef3b1c97-75b21599   
236         236           257  c0ed3dcd-9c7e8e50-067ce990-f4417a65-4c5b379f   
237         237           258  179ffba1-0dbc255c-ec399ad4-962ed53a-a19a0bff   
238         238           259  f1b927dc-db9ba0ef-0e204f32-8ebbc85f-be0fe49a   

                            Unnamed: 1  Participant ID                Name  \
0                                  NaN          3370.0             NICOLAS   
1    no hay PET. Las RM son desde 2020          8964.0        MARIA TERESA   
2                                  NaN         13512.0        MARTA SUSANA   
3                                  NaN         13778.0    LIDIA AIDA NOEMI   
4                                  NaN         15076.0  ALEJANDRA PATRICIA   
..                                 ...             ...                 ...   
234                                NaN        703380.0           FERNANDO    
235                                NaN        712848.0       LIVIA LUCIANA   
236                                NaN        715946.0         PEDRO MARIA   
237                                NaN        717824.0            PATRICIO   
238                                NaN        718566.0   GUILLERMO AUGUSTO   

             Surname                 Nombre T1        AN_MRI    Date_MRI  ...  \
0               RIGA           3D fSPGR-IRprep  9.275000e+12  2013-07-04  ...   
1    BILBAO LA VIEJA                       NaN  9.275001e+12         NaN  ...   
2           EVRANIAN                  FSPGR 3D  9.275000e+12  2015-06-29  ...   
3           WILENSKY   Accelerated Ax IR-FSPGR  9.275001e+12  2019-06-19  ...   
4            FONTANA  Accelerated SAG IR-FSPGR  9.275001e+12  2017-08-04  ...   
..               ...                       ...           ...         ...  ...   
234          BARROS                Ax FSPGR 3D  9.275001e+12  2020-09-11  ...   
235             DEON   Accelerated Ax IR-FSPGR  9.275001e+12  2021-01-08  ...   
236           PAILHE   Accelerated Ax IR-FSPGR  9.275001e+12  2021-02-11  ...   
237           MURPHY               Ax FSPGR 3D  9.275001e+12  2021-05-16  ...   
238          BASILIO   Accelerated Ax IR-FSPGR  9.275001e+12  2021-04-03  ...   

     gctx-bilat-occipital_rel gctx-bilat-occipital_per  \
0                    0.027904                     11.9   
1                         NaN                      NaN   
2                    0.036981                     93.7   
3                    0.027322                      8.9   
4                    0.036672                     83.6   
..                        ...                      ...   
234                  0.018745                      1.0   
235                  0.030842                     44.2   
236                  0.031266                     34.0   
237                  0.023596                      1.0   
238                  0.029793                     24.0   

    gctx-bilat-parietal_abs gctx-bilat-parietal_rel gctx-bilat-parietal_per  \
0              1.334525e+16                0.086964                    54.5   
1                       NaN                     NaN                     NaN   
2              1.157804e+16                0.082528                    35.8   
3              9.642489e+15                0.070050                     1.9   
4              1.333215e+16                0.088888   

In [52]:
csv['Diagnostico Presuntivo']

0      Deterioro cognitivo leve amnÃƒÂ©sico multidominio
1      Deterioro cognitivo leve amnÃƒÂ©sico multidominio
2              Deterioro cognitivo leve amnÃƒÂ©sico puro
3      Deterioro cognitivo leve amnÃƒÂ©sico multidominio
4      Deterioro cognitivo leve amnÃƒÂ©sico multidominio
                             ...                        
234    Deterioro cognitivo leve amnÃƒÂ©sico multidominio
235    Deterioro cognitivo leve amnÃƒÂ©sico multidominio
236    Deterioro cognitivo leve amnÃƒÂ©sico multidominio
237    Deterioro cognitivo leve amnÃƒÂ©sico multidominio
238    Deterioro cognitivo leve multidominio no amnÃƒ...
Name: Diagnostico Presuntivo, Length: 239, dtype: object

In [53]:
csv['Sex']

0      M
1      F
2      F
3      F
4      F
      ..
234    M
235    F
236    M
237    M
238    M
Name: Sex, Length: 239, dtype: object

In [54]:
csv['Age at diagnostic visit (in years)']

0      75.0
1      68.0
2      75.0
3      69.0
4      59.0
       ... 
234    63.0
235    73.0
236    62.0
237    55.0
238    67.0
Name: Age at diagnostic visit (in years), Length: 239, dtype: float64

In [55]:
csv['pet_id']

0      b87479c8-641ec909-8b8effc1-9b58a18a-573f8414
1                                                 -
2      e5a4f8b8-f6ad7d45-52b4b6e8-f3ebf6cd-c8e8f25b
3      c96cbd4d-f9cfda98-8bd71bc1-e26e87e3-e25c6d50
4      b172e084-61708876-5cb503da-b65f4f85-e1c4e110
                           ...                     
234                                               -
235                                               -
236                                               -
237                                               -
238                                               -
Name: pet_id, Length: 239, dtype: object

In [56]:
csv['diag']

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
234    1.0
235    1.0
236    0.0
237    1.0
238    0.0
Name: diag, Length: 239, dtype: float64

In [116]:
csv['anon_id']

0     44a28d4d-715fb1d1-a27b6b74-8453221e-971636d2
1     bed8e34e-9cce115f-41d35acb-2299c2d0-5c1eda47
2     92a4e09d-f3b1bf4b-400c048c-9e0da253-522144d4
3     d3e65f9f-b900bf1c-20711c1d-3c1a2020-b1c08e1d
4     ed140931-163cd49b-c10b164f-f5be07d3-9fdd9020
5     e8c3e0b2-5bc7ec78-36df4fdd-1fd16c31-2de77b5e
6     60afec12-08579f16-432289de-c3a1600c-06562055
7     74cbd572-0a9978dc-25e1ee27-d0291523-a9c6a8b4
8     838b6ced-3e6f25b6-b8799a71-9ab43e5b-ffe8776d
9     164751e1-d6db08f6-8eea8b79-a2ef609c-362a8321
10    898776ca-aef78c5e-e93b96c9-8d4b730b-409da4cf
11    20f18c9a-94af6cfb-41c15927-301a50bb-6574e5b3
12    ed9da20f-0ffdf438-8431ae58-e0f365b8-1c2c968c
13    a2a42680-e0608ab1-2a69fe41-032598e6-1b350022
14    18cd0ce6-a2a728e1-6db1f358-8f85eba2-395d1521
15    dbdcefa2-d37077db-8e21c368-f4fc53a9-16e4751b
16    2db1aabf-bdfd1f90-4bb0ffc1-1a8abf80-742a68d8
17    c5a816aa-9ebed134-583b8cb6-d312d63c-e763b44c
18    6d955f3c-d72106fa-9acfcafa-9788d41b-95a19124
19    6679c9ac-ff5b3eee-73520f1

In [8]:
df = csv.reset_index()  # make sure indexes pair with number of rows

df2 = pd.DataFrame()

len(df2)

for index, row in df.iterrows():
    petId = str(row['pet_id'])
    path = '/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/' + petId
    if os.path.exists(path):
        print(path)
        df2 = df2.append(row)
    
len(df2)

/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/b87479c8-641ec909-8b8effc1-9b58a18a-573f8414
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/e5a4f8b8-f6ad7d45-52b4b6e8-f3ebf6cd-c8e8f25b
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/c96cbd4d-f9cfda98-8bd71bc1-e26e87e3-e25c6d50
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/b172e084-61708876-5cb503da-b65f4f85-e1c4e110
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/ad80ea14-f27dad6f-3e12c1f0-2993bcc3-4cb99a80
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/01959b9f-793181a7-043f0671-0c14bc59-edf43b59
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/c228c496-01ac9239-b1218004-d7d296e5-9c9c1248
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/47b78623-8a9948f9-ad801210-dc3262e3-d2f9343e
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/c27f0f13-eddbc624-7692d026-310587e3-15a7b566
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/ccbec475-d4eccc2e-e8064ee

/var/folders/hy/rbnyh5md5yvc72ynxb0c2cz40000gn/T/ipykernel_36283/2174592208.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(row)
/var/folders/hy/rbnyh5md5yvc72ynxb0c2cz40000gn/T/ipykernel_36283/2174592208.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(row)
/var/folders/hy/rbnyh5md5yvc72ynxb0c2cz40000gn/T/ipykernel_36283/2174592208.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(row)
/var/folders/hy/rbnyh5md5yvc72ynxb0c2cz40000gn/T/ipykernel_36283/2174592208.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(row)
/var/folders/hy/rbnyh5md5yvc72ynxb0c2cz4

/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/de73b3c8-caff4051-ac0df0b6-fe691283-8d5a2f43
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/1ecd8590-a4b58359-30292d24-bc850d05-39d0d4e9
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/faf3bc8b-bca09175-6489651a-14790247-226512a2
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/65c3a537-c34284b3-a83d7952-02163561-16114f0c
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/3656419d-1523529f-f080cc08-ac5fbbf1-441422c8
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/3f3c0b5a-e6630b26-53079866-423b2546-b61f03e4
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/c19aae90-3931bfc8-9bc8fbfc-3b6bb72c-d78aba7b
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/51384f71-9b488f68-cad638e8-3f8995e6-5f05ddfe
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/a248637a-74b50589-b5ec9433-91d05f99-2cb3100d
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/3eb85eee-8babc6d8-8b32a24

/var/folders/hy/rbnyh5md5yvc72ynxb0c2cz40000gn/T/ipykernel_36283/2174592208.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(row)
/var/folders/hy/rbnyh5md5yvc72ynxb0c2cz40000gn/T/ipykernel_36283/2174592208.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(row)
/var/folders/hy/rbnyh5md5yvc72ynxb0c2cz40000gn/T/ipykernel_36283/2174592208.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(row)
/var/folders/hy/rbnyh5md5yvc72ynxb0c2cz40000gn/T/ipykernel_36283/2174592208.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(row)
/var/folders/hy/rbnyh5md5yvc72ynxb0c2cz4

/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/ac6d0a1e-e9d1ac88-1b2feb1c-21990129-b2738ead
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/7861b0e1-238cb043-5de0d771-e9ef9375-25de44c0
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/32bad7a9-d205e9c0-c68eec0d-5c4ca07c-839e4dae
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/ad80ea14-f27dad6f-3e12c1f0-2993bcc3-4cb99a80
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/427bf2d0-eed4925a-8a094d07-93f5e700-8da3bc5b
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/a61f65bd-153b85c6-10b27604-ec7125c5-b800a82c
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/4f9d582a-c992d94b-6418c65a-8866c929-7a52d395
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/22e42fbe-2f5e8375-337266d7-9610b183-a923614e
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/0b010b72-4a40a743-412c507d-7c75f235-6e6b3ffb
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/ae4130ac-1c57b6ba-1087205

...

/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/98e82e34-df3aa054-551be0aa-6ff6d7c9-6ce792a9
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/6559a881-0fc99b85-c04b11f4-d7843d40-6d628caa
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/99f955dc-0188be2f-5b206466-cb05b629-6583c4ba
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/026fadeb-4ac5192a-7baa727a-fef84e8b-f58748ec
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/d0fcd3bf-a0f2ef64-2f7931ec-28e68412-c0e2a63a
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/6b7e72b1-7ba09544-faf79346-091fd381-88d09293
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/d1bd3533-83d0ac7b-ffecb960-b7f6d96a-75cbbca7
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/a33eb1ba-4a5092e1-69d9b29a-2e3a10a2-91914a92
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/9fa9589f-30f1da5a-c1441402-c97efd2e-c5af6926
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/3ff386f7-356e5430-da77c39

/var/folders/hy/rbnyh5md5yvc72ynxb0c2cz40000gn/T/ipykernel_36283/2174592208.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(row)
/var/folders/hy/rbnyh5md5yvc72ynxb0c2cz40000gn/T/ipykernel_36283/2174592208.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(row)
/var/folders/hy/rbnyh5md5yvc72ynxb0c2cz40000gn/T/ipykernel_36283/2174592208.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(row)
/var/folders/hy/rbnyh5md5yvc72ynxb0c2cz40000gn/T/ipykernel_36283/2174592208.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(row)
/var/folders/hy/rbnyh5md5yvc72ynxb0c2cz4

/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/24fa2c1d-1218de8c-c96d25cb-28e77c7a-89e0c0a4
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/1541a4b6-f7f308c1-9e017a0a-d247bc16-e3117067
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/5e28bc57-30e1abd2-a0a7d856-4fc1af1e-5bee10d2
/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed3/625ddb3f-6b6adb4a-0653c1b1-1e566cc8-3fe2fab5


/var/folders/hy/rbnyh5md5yvc72ynxb0c2cz40000gn/T/ipykernel_36283/2174592208.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(row)
/var/folders/hy/rbnyh5md5yvc72ynxb0c2cz40000gn/T/ipykernel_36283/2174592208.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(row)


115

In [9]:
df2['diag']

0      1.0
2      1.0
3      1.0
4      1.0
6      0.0
      ... 
225    1.0
226    1.0
231    1.0
232    1.0
233    1.0
Name: diag, Length: 115, dtype: float64

In [10]:
duplicate_rows = df2[df2.duplicated('pet_id', keep=False)]

# Display the rows with repeated pet_id values
duplicate_rows[['pet_id', 'anon_id', 'diag', 'Name', 'Surname', 'Age at diagnostic visit (in years)']].sort_values(by=['pet_id'])

,pet_id,anon_id,diag,Name,Surname,Age at diagnostic visit (in years)
20,38471673-ac5b7bce-f377e18a-2bce64ba-15a7f769,4e509147-8b314f62-ace7bd77-aa0113a8-c59aaada,1.0,HEBE MARIA,RODRIGUEZ,81.0
104,38471673-ac5b7bce-f377e18a-2bce64ba-15a7f769,4e509147-8b314f62-ace7bd77-aa0113a8-c59aaada,1.0,OSCAR ESTANISLAO,RODRIGUEZ,75.0
41,7610266b-ca235fd1-d02b147f-e658c7e4-ac52c728,6bd9f5c3-cf85f26c-85358d1f-58e3ab3a-7e723981,1.0,GENOVEVA ANGELICA,ALVAREZ,77.0
73,7610266b-ca235fd1-d02b147f-e658c7e4-ac52c728,6591cf51-7cce7688-80e7e58f-e8581f03-3766e111,1.0,ALBERTO MANUEL,ALVAREZ GAIANI,75.0
176,7610266b-ca235fd1-d02b147f-e658c7e4-ac52c728,6bd9f5c3-cf85f26c-85358d1f-58e3ab3a-7e723981,0.0,OSCAR GUILLERMO,ALVAREZ,66.0
6,ad80ea14-f27dad6f-3e12c1f0-2993bcc3-4cb99a80,82492442-313797f9-20456fb1-e93414ad-69e5d284,0.0,ALBERTO RAMON,LOPEZ,92.0
134,ad80ea14-f27dad6f-3e12c1f0-2993bcc3-4cb99a80,82492442-313797f9-20456fb1-e93414ad-69e5d284,1.0,SALUSTIANA,LOPEZ,70.0
172,ad80ea14-f27dad6f-3e12c1f0-2993bcc3-4cb99a80,82492442-313797f9-20456fb1-e93414ad-69e5d284,1.0,ELENA,LOPEZ,62.0
77,caa1ef27-21bb54d0-05caa39b-0fce0ae2-4218d1cc,86d28782-a694a412-7fa3da61-e77e069b-b100fdc5,1.0,RAUL ALFREDO,VILLANUEVA,59.0
173,caa1ef27-21bb54d0-05caa39b-0fce0ae2-4218d1cc,86d28782-a694a412-7fa3da61-e77e069b-b100fdc5,0.0,RICARDO LUIS,VILLANUEVA,75.0


In [13]:
# curated, no tiene duplicados, etc.
df3 = pd.read_csv('/Users/hugom/PET-IA/Sets/fleni-myriam-curated.csv')
df3

,Unnamed: 0,diag,pet_id,Sex,Age at diagnostic visit (in years),anon_id
0,0,1.0,b87479c8-641ec909-8b8effc1-9b58a18a-573f8414,M,75.0,601894e9-7192ca44-617903f9-b7775028-b7db3c01
1,2,1.0,e5a4f8b8-f6ad7d45-52b4b6e8-f3ebf6cd-c8e8f25b,F,75.0,26f052fa-48d0929f-43515de0-ab857928-4ee712ec
2,3,1.0,c96cbd4d-f9cfda98-8bd71bc1-e26e87e3-e25c6d50,F,69.0,9c673f58-f5bfbaf0-ff226082-410ef24e-e7d8ff4a
3,4,1.0,b172e084-61708876-5cb503da-b65f4f85-e1c4e110,F,59.0,0f5ab5b2-c7051e97-42563753-f7f14410-29b463dd
4,7,0.0,01959b9f-793181a7-043f0671-0c14bc59-edf43b59,M,76.0,c0f0fdd2-3d2bda20-ca3c99c1-154d11f1-4f7d8346
...,...,...,...,...,...,...
95,225,1.0,a4dc5f48-55b6fe7b-95df4cc2-d1a9765f-acf114ce,F,64.0,b5b58aa6-ac714391-3e93da6a-b4f7a79e-7f5df46e
96,226,1.0,24fa2c1d-1218de8c-c96d25cb-28e77c7a-89e0c0a4,M,66.0,8810c5bf-2dfa6391-82841f0b-e271e5da-d8256c22
97,231,1.0,1541a4b6-f7f308c1-9e017a0a-d247bc16-e3117067,M,67.0,d1b48cde-e02de145-fcf116ba-ec2485a2-2731e736
98,232,1.0,5e28bc57-30e1abd2-a0a7d856-4fc1af1e-5bee10d2,M,82.0,208ef863-72c22ba9-681a25e4-799847e4-cb2cdb64


Hay dos pets que tienen distinto diagnóstico, deberían ser eliminadas?

In [14]:
df4 = df3.groupby(by=['Sex', 'diag'])

In [15]:
df4.describe()

Unnamed: 0                                                      \
              count        mean        std   min     25%    50%     75%   
Sex diag                                                                  
F   0.0        16.0   90.000000  64.662199  18.0   37.75   72.5  152.00   
    1.0        27.0  114.074074  76.075330   2.0   34.00  142.0  174.00   
M   0.0        24.0  127.166667  75.342503   7.0   60.25  149.5  193.75   
    1.0        33.0  157.757576  63.096667   0.0  133.00  167.0  205.00   

                Age at diagnostic visit (in years)                        \
            max                              count       mean        std   
Sex diag                                                                   
F   0.0   220.0                               16.0  73.187500   7.268368   
    1.0   225.0                               27.0  73.481481   7.757728   
M   0.0   219.0                               24.0  69.291667  10.246862   
    1.0   233.0                               33.0  74.969697   7.588992   

                                         
           min   25%   50%    75%   max  
Sex diag                                 
F   0.0   55.0  70.5  74.0  77.25  87.0  
    1.0   59.0  68.0  74.0  79.50  90.0  
M   0.0   43.0  65.5  70.5  76.50  82.0  
    1.0   55.0  72.0  77.0  80.00  86.0

In [16]:
df3.groupby(by='Sex').describe()

Unnamed: 0                                                         diag  \
         count        mean        std  min   25%    50%    75%    max count   
Sex                                                                           
F         43.0  105.116279  72.212388  2.0  36.5  109.0  165.0  225.0  43.0   
M         57.0  144.877193  69.559859  0.0  96.0  161.0  199.0  233.0  57.0   

               ...           Age at diagnostic visit (in years)             \
         mean  ...  75%  max                              count       mean   
Sex            ...                                                           
F    0.627907  ...  1.0  1.0                               43.0  73.372093   
M    0.578947  ...  1.0  1.0                               57.0  72.578947   

                                             
          std   min   25%   50%   75%   max  
Sex                                          
F    7.492928  55.0  68.5  74.0  78.5  90.0  
M    9.166997  43.0  67.0  76.0  79.0  86.0  

[2 rows x 24 columns]

In [17]:
df3['Age at diagnostic visit (in years)'].describe()

count    100.000000
mean      72.920000
std        8.456281
min       43.000000
25%       68.000000
50%       75.000000
75%       79.000000
max       90.000000
Name: Age at diagnostic visit (in years), dtype: float64

# Fleni60

In [112]:
csv = pd.read_csv('/Users/hugom/PET-IA/Sets/fleni-PET_clasificados60.csv')

In [113]:
duplicate_rows = csv[csv.duplicated('anon_id', keep=False)]
duplicate_rows

,StudyDate,anon_id,PatientSex,Unnamed: 3,Conclusion PET


Este no tiene duplicados, todo OK

In [114]:
csv2 = csv.groupby(by=['PatientSex', 'Conclusion PET'])

In [115]:
csv2.describe()

StudyDate                                         \
                              count          mean          std         min   
PatientSex Conclusion PET                                                    
F          AD                  18.0  2.022082e+07   319.814616  20220210.0   
           non-AD              10.0  2.022082e+07   201.919016  20220628.0   
M          AD                  21.0  2.022069e+07   350.283504  20220217.0   
           non-AD              11.0  2.022178e+07  2771.629731  20220622.0   

                                                                             \
                                   25%         50%          75%         max   
PatientSex Conclusion PET                                                     
F          AD              20220535.75  20220859.5  20221026.75  20221216.0   
           non-AD          20220648.50  20220809.5  20220883.25  20221216.0   
M          AD              20220419.00  20220614.0  20221021.00  20221227.0   
           non-AD          20220758.00  20221025.0  20221224.50  20230105.0   

                          Unnamed: 3                               
                               count mean std min 25% 50% 75% max  
PatientSex Conclusion PET                                          
F          AD                    0.0  NaN NaN NaN NaN NaN NaN NaN  
           non-AD                0.0  NaN NaN NaN NaN NaN NaN NaN  
M          AD                    0.0  NaN NaN NaN NaN NaN NaN NaN  
           non-AD                0.0  NaN NaN NaN NaN NaN NaN NaN

# Otra data

In [4]:
df = pd.read_csv("/Users/hugom/PET-IA/Data/ADNI-FULL-PostProc_3_01_2023_UniformResolution.csv")

In [7]:
df['Acq Date'].min()

'1/03/2007'

In [8]:
df['Acq Date'].max()

'9/30/2019'

In [9]:
dfs = df.groupby(by='Subject')

In [11]:
dfs['Acq Date'].mean()

TypeError: Could not convert 6/09/2011 to numeric

# Mérida

In [4]:
tsvFile = '/Users/hugom/PET-IA/Full-DBs/ines-merida-db/pool/DM/TEP/CERMEP_MXFDG/BASE/DATABASE_SENT/ALL/participants.tsv'
csv = pd.read_table(tsvFile, index_col = False)

In [5]:
csv

,participant_id,birth_date,age,sex,weight,size,injected_dose,activity_in_ICV_mask,handedness,comments
0,sub-0001,1980,35,F,81,163.0,149,6911,R,NaN
1,sub-0002,1957,58,F,66,160.0,102,5781,R,NaN
2,sub-0003,1979,36,M,88,174.0,144,5980,R,NaN
3,sub-0004,1963,51,M,72,NaN,124,6551,R,NaN
4,sub-0005,1981,33,M,110,180.0,147,4925,R,NaN
5,sub-0006,1974,41,M,76,176.0,119,5590,R,NaN
6,sub-0007,1975,40,F,60,170.0,118,8271,R,NaN
7,sub-0008,1988,27,F,53,NaN,95,6889,R,NaN
8,sub-0009,1986,28,F,66,NaN,135,7974,R,NaN
9,sub-0010,1971,43,M,117,170.0,168,6137,R,NaN


In [8]:
csv.groupby(by='sex')['age'].describe()

,count,mean,std,min,25%,50%,75%,max
sex,,,,,,,,
F,20.0,39.9,12.652855,24.0,27.75,37.5,48.5,65.0
M,17.0,36.0,9.565563,23.0,27.00,34.0,42.0,56.0


In [9]:
csv['age'].describe()

count    37.000000
mean     38.108108
std      11.359733
min      23.000000
25%      27.000000
50%      35.000000
75%      46.000000
max      65.000000
Name: age, dtype: float64

# Oasis 3

In [9]:
csvDiagnoses = pd.read_csv("/Users/hugom/Downloads/OASIS3_data_files/scans/UDSd1-Form_D1__Clinician_Diagnosis___Cognitive_Status_and_Dementia/resources/csv/files/OASIS3_UDSd1_diagnoses.csv")
csvOwn = pd.read_csv("/Users/hugom/PET-IA/Sets/oasis3.csv")
csvCN = pd.read_csv("/Users/hugom/Downloads/OASIS_cohort_files/scans/CogNorm-Cognitively_Normal_Cohorts/resources/csv/files/OASIS3_unchanged_CDR_cognitively_healthy.csv")

In [10]:
csvOwn

,StudyID,OASISID,days_to_visit,matched_diagnosis_session_label,Group,Sex,Age
0,sub-OAS30001_ses-d2650_acq-FDG_pet,NaN,NaN,NaN,CN,NaN,72.45
1,sub-OAS30003_ses-d3320_acq-FDG_pet,NaN,NaN,NaN,CN,NaN,67.90
2,sub-OAS30007_ses-d1981_acq-FDG_pet,NaN,NaN,NaN,NaN,NaN,77.01
3,sub-OAS30032_ses-d3511_acq-FDG_pet,NaN,NaN,NaN,CN,NaN,59.79
4,sub-OAS30032_ses-d4121_acq-FDG_pet,NaN,NaN,NaN,CN,NaN,61.47
...,...,...,...,...,...,...,...
121,sub-OAS31114_ses-d3338_acq-FDG_pet,NaN,NaN,NaN,CN,NaN,NaN
122,sub-OAS31118_ses-d2864_acq-FDG_pet,NaN,NaN,NaN,CN,NaN,NaN
123,sub-OAS31153_ses-d1570_acq-FDG_pet,NaN,NaN,NaN,CN,NaN,NaN
124,sub-OAS31165_ses-d0990_acq-FDG_pet,NaN,NaN,NaN,CN,NaN,NaN


# Primero, matcheamos todo

TODO: mover a otro archivo

In [68]:
csvCN

,OASIS3_id,Min of CDRTOT,Max of CDRTOT
0,OAS30001,0,0
1,OAS30002,0,0
2,OAS30003,0,0
3,OAS30004,0,0
4,OAS30005,0,0
...,...,...,...
750,OAS31465,0,0
751,OAS31469,0,0
752,OAS31470,0,0
753,OAS31471,0,0


In [69]:
csvOwn["Group"] = None

In [70]:
csvOwn

,StudyID,OASISID,days_to_visit,matched_diagnosis_session_label,Group,Sex,Age
0,sub-OAS30001_ses-d2650_acq-FDG_pet,OAS30001,2650,OAS30001_UDSd1_d2699,None,NaN,72.45
1,sub-OAS30003_ses-d3320_acq-FDG_pet,OAS30003,3320,OAS30003_UDSd1_d3288,None,NaN,67.90
2,sub-OAS30007_ses-d1981_acq-FDG_pet,OAS30007,1981,OAS30007_UDSd1_d1935,None,NaN,77.01
3,sub-OAS30032_ses-d3511_acq-FDG_pet,OAS30032,3511,OAS30032_UDSd1_d3387,None,NaN,59.79
4,sub-OAS30032_ses-d4121_acq-FDG_pet,OAS30032,4121,OAS30032_UDSd1_d4756,None,NaN,61.47
...,...,...,...,...,...,...,...
121,sub-OAS31114_ses-d3338_acq-FDG_pet,OAS31114,3338,OAS31114_UDSd1_d3305,None,NaN,NaN
122,sub-OAS31118_ses-d2864_acq-FDG_pet,OAS31118,2864,OAS31118_UDSd1_d2764,None,NaN,NaN
123,sub-OAS31153_ses-d1570_acq-FDG_pet,OAS31153,1570,OAS31153_UDSd1_d1478,None,NaN,NaN
124,sub-OAS31165_ses-d0990_acq-FDG_pet,OAS31165,990,OAS31165_UDSd1_d1072,None,NaN,NaN


In [71]:
csvDiagnoses

,OASISID,OASIS_session_label,days_to_visit,age at visit,WHODIDDX,NORMCOG,DEMENTED,MCIAMEM,MCIAPLUS,MCIAPLAN,...,schizoif,anxiet,anxietif,delir,delirif,ptsddx,ptsddxif,alcabuse,impsub,impsubif
0,OAS30001,OAS30001_UDSd1_d0000,0,65.19,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OAS30001,OAS30001_UDSd1_d0339,339,66.12,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OAS30001,OAS30001_UDSd1_d0722,722,67.17,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OAS30001,OAS30001_UDSd1_d1106,1106,68.22,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OAS30001,OAS30001_UDSd1_d1456,1456,69.18,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8494,OAS31472,OAS31472_UDSd1_d0826,826,69.53,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN
8495,OAS31473,OAS31473_UDSd1_d0000,0,56.61,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN
8496,OAS31473,OAS31473_UDSd1_d1142,1142,59.74,NaN,0.0,0.0,1.0,0.0,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN
8497,OAS31474,OAS31474_UDSd1_d0000,0,81.84,NaN,0.0,0.0,1.0,0.0,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN


In [72]:
def getDaysToVisit(studyID):
    """
    Devuelve los días de una visita desde el nombre de una imágen (StudyID)
    """
    search = re.search('-d(\d*)_acq', studyID)
    return int(search[1])
print(getDaysToVisit("sub-OAS30032_ses-d4121_acq-FDG_pet"))

4121


In [73]:
def getOASISID(StudyID):
    """
    Devuelve el OASISID a partir del StudyID (Campo inventado por mi)
    """
    search = re.search('(OAS\d*)_', StudyID)
    return search[1]
print(getOASISID("OAS31473_UDSd1_d1142"))

OAS31473


In [74]:
def closerSession(row, csvDiagnoses):
    """
    Devuelve la sesión "mas cercana" para una row
    """
    daysToVisit = getDaysToVisit(row['StudyID'])
    oasisID = getOASISID(row['StudyID'])
    csv2 = csvDiagnoses[csvDiagnoses['OASISID'] ==  oasisID]
    closerSessionID = None
    closerDaysToVisit = math.inf
    # De aquí debe devolver la sesión mas "cercana"
    for index, row in csv2.iterrows():
        diagnosisDaysToVisit = row['days_to_visit']
        diffInDays = abs(abs(diagnosisDaysToVisit) - abs(daysToVisit))
        if diffInDays < closerDaysToVisit:
            closerDaysToVisit = diffInDays
            closerSessionID = row['OASIS_session_label']
            
    return closerSessionID

In [75]:
csvOwn["matched_diagnosis_session_label"] = csvOwn.apply(lambda row: closerSession(row, csvDiagnoses), axis = 1)

In [76]:
csvOwn

,StudyID,OASISID,days_to_visit,matched_diagnosis_session_label,Group,Sex,Age
0,sub-OAS30001_ses-d2650_acq-FDG_pet,OAS30001,2650,OAS30001_UDSd1_d2699,None,NaN,72.45
1,sub-OAS30003_ses-d3320_acq-FDG_pet,OAS30003,3320,OAS30003_UDSd1_d3288,None,NaN,67.90
2,sub-OAS30007_ses-d1981_acq-FDG_pet,OAS30007,1981,OAS30007_UDSd1_d1935,None,NaN,77.01
3,sub-OAS30032_ses-d3511_acq-FDG_pet,OAS30032,3511,OAS30032_UDSd1_d3387,None,NaN,59.79
4,sub-OAS30032_ses-d4121_acq-FDG_pet,OAS30032,4121,OAS30032_UDSd1_d4756,None,NaN,61.47
...,...,...,...,...,...,...,...
121,sub-OAS31114_ses-d3338_acq-FDG_pet,OAS31114,3338,OAS31114_UDSd1_d3305,None,NaN,NaN
122,sub-OAS31118_ses-d2864_acq-FDG_pet,OAS31118,2864,OAS31118_UDSd1_d2764,None,NaN,NaN
123,sub-OAS31153_ses-d1570_acq-FDG_pet,OAS31153,1570,OAS31153_UDSd1_d1478,None,NaN,NaN
124,sub-OAS31165_ses-d0990_acq-FDG_pet,OAS31165,990,OAS31165_UDSd1_d1072,None,NaN,NaN


In [77]:
csvOwn["days_to_visit"] = csvOwn.apply(lambda row: getDaysToVisit(row["StudyID"]), axis = 1)

In [78]:
csvOwn["OASISID"] = csvOwn.apply(lambda row: getOASISID(row["StudyID"]), axis = 1)

In [79]:
csvOwn

,StudyID,OASISID,days_to_visit,matched_diagnosis_session_label,Group,Sex,Age
0,sub-OAS30001_ses-d2650_acq-FDG_pet,OAS30001,2650,OAS30001_UDSd1_d2699,None,NaN,72.45
1,sub-OAS30003_ses-d3320_acq-FDG_pet,OAS30003,3320,OAS30003_UDSd1_d3288,None,NaN,67.90
2,sub-OAS30007_ses-d1981_acq-FDG_pet,OAS30007,1981,OAS30007_UDSd1_d1935,None,NaN,77.01
3,sub-OAS30032_ses-d3511_acq-FDG_pet,OAS30032,3511,OAS30032_UDSd1_d3387,None,NaN,59.79
4,sub-OAS30032_ses-d4121_acq-FDG_pet,OAS30032,4121,OAS30032_UDSd1_d4756,None,NaN,61.47
...,...,...,...,...,...,...,...
121,sub-OAS31114_ses-d3338_acq-FDG_pet,OAS31114,3338,OAS31114_UDSd1_d3305,None,NaN,NaN
122,sub-OAS31118_ses-d2864_acq-FDG_pet,OAS31118,2864,OAS31118_UDSd1_d2764,None,NaN,NaN
123,sub-OAS31153_ses-d1570_acq-FDG_pet,OAS31153,1570,OAS31153_UDSd1_d1478,None,NaN,NaN
124,sub-OAS31165_ses-d0990_acq-FDG_pet,OAS31165,990,OAS31165_UDSd1_d1072,None,NaN,NaN


In [80]:
merged = csvOwn.merge(
    csvDiagnoses,
    left_on = ["matched_diagnosis_session_label"],
    right_on = ["OASIS_session_label"],
    how = "inner",
    copy = True
)

In [81]:
merged

,StudyID,OASISID_x,days_to_visit_x,matched_diagnosis_session_label,Group,Sex,Age,OASISID_y,OASIS_session_label,days_to_visit_y,...,schizoif,anxiet,anxietif,delir,delirif,ptsddx,ptsddxif,alcabuse,impsub,impsubif
0,sub-OAS30001_ses-d2650_acq-FDG_pet,OAS30001,2650,OAS30001_UDSd1_d2699,None,NaN,72.45,OAS30001,OAS30001_UDSd1_d2699,2699,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sub-OAS30003_ses-d3320_acq-FDG_pet,OAS30003,3320,OAS30003_UDSd1_d3288,None,NaN,67.90,OAS30003,OAS30003_UDSd1_d3288,3288,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sub-OAS30007_ses-d1981_acq-FDG_pet,OAS30007,1981,OAS30007_UDSd1_d1935,None,NaN,77.01,OAS30007,OAS30007_UDSd1_d1935,1935,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sub-OAS30032_ses-d3511_acq-FDG_pet,OAS30032,3511,OAS30032_UDSd1_d3387,None,NaN,59.79,OAS30032,OAS30032_UDSd1_d3387,3387,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sub-OAS30032_ses-d4121_acq-FDG_pet,OAS30032,4121,OAS30032_UDSd1_d4756,None,NaN,61.47,OAS30032,OAS30032_UDSd1_d4756,4756,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,sub-OAS31114_ses-d3338_acq-FDG_pet,OAS31114,3338,OAS31114_UDSd1_d3305,None,NaN,NaN,OAS31114,OAS31114_UDSd1_d3305,3305,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,sub-OAS31118_ses-d2864_acq-FDG_pet,OAS31118,2864,OAS31118_UDSd1_d2764,None,NaN,NaN,OAS31118,OAS31118_UDSd1_d2764,2764,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,sub-OAS31153_ses-d1570_acq-FDG_pet,OAS31153,1570,OAS31153_UDSd1_d1478,None,NaN,NaN,OAS31153,OAS31153_UDSd1_d1478,1478,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,sub-OAS31165_ses-d0990_acq-FDG_pet,OAS31165,990,OAS31165_UDSd1_d1072,None,NaN,NaN,OAS31165,OAS31165_UDSd1_d1072,1072,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN


In [82]:
merged.columns

Index(['StudyID', 'OASISID_x', 'days_to_visit_x',
       'matched_diagnosis_session_label', 'Group', 'Sex', 'Age', 'OASISID_y',
       'OASIS_session_label', 'days_to_visit_y',
       ...
       'schizoif', 'anxiet', 'anxietif', 'delir', 'delirif', 'ptsddx',
       'ptsddxif', 'alcabuse', 'impsub', 'impsubif'],
      dtype='object', length=156)

In [83]:
len(merged[merged["NORMCOG"] == 1])

114

In [84]:
normalCSV = merged[merged["NORMCOG"] == 1]

In [85]:
normalCSV.to_csv("/Users/hugom/PET-IA/Sets/oasis3-cognitively-normal.csv")

In [86]:
nonNormal = merged[merged["NORMCOG"] == 0]

In [87]:
len(nonNormal)

11

In [88]:
nonNormal

,StudyID,OASISID_x,days_to_visit_x,matched_diagnosis_session_label,Group,Sex,Age,OASISID_y,OASIS_session_label,days_to_visit_y,...,schizoif,anxiet,anxietif,delir,delirif,ptsddx,ptsddxif,alcabuse,impsub,impsubif
22,sub-OAS30194_ses-d9317_acq-FDG_pet,OAS30194,9317,OAS30194_UDSd1_d9229,None,NaN,NaN,OAS30194,OAS30194_UDSd1_d9229,9229,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,sub-OAS30194_ses-d9874_acq-FDG_pet,OAS30194,9874,OAS30194_UDSd1_d9938,None,NaN,NaN,OAS30194,OAS30194_UDSd1_d9938,9938,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN
26,sub-OAS30241_ses-d3607_acq-FDG_pet,OAS30241,3607,OAS30241_UDSd1_d3598,None,NaN,NaN,OAS30241,OAS30241_UDSd1_d3598,3598,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,sub-OAS30403_ses-d2758_acq-FDG_pet,OAS30403,2758,OAS30403_UDSd1_d2277,None,NaN,NaN,OAS30403,OAS30403_UDSd1_d2277,2277,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,sub-OAS30452_ses-d4489_acq-FDG_pet,OAS30452,4489,OAS30452_UDSd1_d4431,None,NaN,NaN,OAS30452,OAS30452_UDSd1_d4431,4431,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN
71,sub-OAS30713_ses-d1853_acq-FDG_pet,OAS30713,1853,OAS30713_UDSd1_d1883,None,NaN,NaN,OAS30713,OAS30713_UDSd1_d1883,1883,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,sub-OAS30717_ses-d2608_acq-FDG_pet,OAS30717,2608,OAS30717_UDSd1_d2460,None,NaN,NaN,OAS30717,OAS30717_UDSd1_d2460,2460,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,sub-OAS30775_ses-d3471_acq-FDG_pet,OAS30775,3471,OAS30775_UDSd1_d3284,None,NaN,NaN,OAS30775,OAS30775_UDSd1_d3284,3284,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN
101,sub-OAS30899_ses-d2471_acq-FDG_pet,OAS30899,2471,OAS30899_UDSd1_d2298,None,NaN,NaN,OAS30899,OAS30899_UDSd1_d2298,2298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,sub-OAS30959_ses-d3174_acq-FDG_pet,OAS30959,3174,OAS30959_UDSd1_d3314,None,NaN,NaN,OAS30959,OAS30959_UDSd1_d3314,3314,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
nonNormal[nonNormal["DEMENTED"] == 1]

,StudyID,OASISID_x,days_to_visit_x,matched_diagnosis_session_label,Group,Sex,Age,OASISID_y,OASIS_session_label,days_to_visit_y,...,schizoif,anxiet,anxietif,delir,delirif,ptsddx,ptsddxif,alcabuse,impsub,impsubif
23,sub-OAS30194_ses-d9874_acq-FDG_pet,OAS30194,9874,OAS30194_UDSd1_d9938,None,NaN,NaN,OAS30194,OAS30194_UDSd1_d9938,9938,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN
101,sub-OAS30899_ses-d2471_acq-FDG_pet,OAS30899,2471,OAS30899_UDSd1_d2298,None,NaN,NaN,OAS30899,OAS30899_UDSd1_d2298,2298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
nonNormal[nonNormal["amndem"] == 1]

,StudyID,OASISID_x,days_to_visit_x,matched_diagnosis_session_label,Group,Sex,Age,OASISID_y,OASIS_session_label,days_to_visit_y,...,schizoif,anxiet,anxietif,delir,delirif,ptsddx,ptsddxif,alcabuse,impsub,impsubif
23,sub-OAS30194_ses-d9874_acq-FDG_pet,OAS30194,9874,OAS30194_UDSd1_d9938,None,NaN,NaN,OAS30194,OAS30194_UDSd1_d9938,9938,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN


In [91]:
nonNormal.to_csv("/Users/hugom/PET-IA/Sets/oasis3-cognitively-anormal.csv")

# Fleni600

In [2]:
fleni600 = pd.read_csv("/Users/hugom/PET-IA/Full-DBs/fleni600.csv")

In [6]:
fleni600

,Unnamed: 0,StudyDate,anon_id,PatientID,PatientSex,PatientName,Conclusion,Diagnóstico
0,238,20200825,625ddb3f-6b6adb4a-0653c1b1-1e566cc8-3fe2fab5,702774,M,WANGER^WALTER ROBERTO,-ALa distribución espacial de los hallazgos ...,AD
1,234,20201006,c45568f2-c860c13b-ddabe3eb-7bc72f65-dc96ba33,701575,F,RUBIO^MARIA ALEJANDRA,-ALa distribución espacial de los hallazgos ...,AD
2,435,20190426,b0e1b100-f29d020b-07fb9c16-bb41abdc-43d7b0e8,597445,F,BRALO^ALICIA ESTER,-ALa distribución espacial de los hallazgos ...,AD
3,247,20200804,ee1769c5-189b12d7-187a454e-f13189f4-19f82052,554900,M,LOBOS^ANIBAL PEDRO,Amiloidosis cortical difusa en el presente e...,AD
4,387,20190718,ec84b311-4f89bd80-bfd691ed-bb1cd2f4-c700b95c,655037,F,SANCHEZ^MARCELA ADRIANA,Amiloidosis cortical difusa en el presente e...,AD
...,...,...,...,...,...,...,...,...
1610,1610,20130215,394f9c0f-407407df-924d39d5-385898a0-103e5c19,474496,M,RIVAROLA^HUGO EUGENIO,NaN,NaN
1611,1611,20130221,2137659d-aa1d903d-beb5a549-8d252a72-a903e6a2,266991,F,BER^PATRICIA,NaN,NaN
1612,1612,20130227,291a10ba-ca622d77-7eb09271-f0948eeb-745ef704,475954,M,GALOSI JUAN CARLOS,NaN,NaN
1613,1613,20130205,667cb287-d9eee39e-e4a18254-beebf4c8-52aa0723,188881,F,VENANCIO^MARIA ALEJANDRA,NaN,NaN


In [3]:
fleni600.describe()

,Unnamed: 0,StudyDate
count,1615.000000,1.615000e+03
mean,807.000000,2.017275e+07
std,466.354658,2.851045e+04
min,0.000000,2.011102e+07
25%,403.500000,2.015053e+07
50%,807.000000,2.017081e+07
75%,1210.500000,2.019117e+07
max,1614.000000,2.023021e+07


In [4]:
fleni600.head()

,Unnamed: 0,StudyDate,anon_id,PatientID,PatientSex,PatientName,Conclusion,Diagnóstico
0,238,20200825,625ddb3f-6b6adb4a-0653c1b1-1e566cc8-3fe2fab5,702774,M,WANGER^WALTER ROBERTO,-ALa distribución espacial de los hallazgos ...,AD
1,234,20201006,c45568f2-c860c13b-ddabe3eb-7bc72f65-dc96ba33,701575,F,RUBIO^MARIA ALEJANDRA,-ALa distribución espacial de los hallazgos ...,AD
2,435,20190426,b0e1b100-f29d020b-07fb9c16-bb41abdc-43d7b0e8,597445,F,BRALO^ALICIA ESTER,-ALa distribución espacial de los hallazgos ...,AD
3,247,20200804,ee1769c5-189b12d7-187a454e-f13189f4-19f82052,554900,M,LOBOS^ANIBAL PEDRO,Amiloidosis cortical difusa en el presente e...,AD
4,387,20190718,ec84b311-4f89bd80-bfd691ed-bb1cd2f4-c700b95c,655037,F,SANCHEZ^MARCELA ADRIANA,Amiloidosis cortical difusa en el presente e...,AD


In [12]:
len(fleni600["PatientID"].unique())

1419

In [13]:
len(fleni600["anon_id"].unique())

1615

In [14]:
fleni600['Diagnóstico'].describe()

count     638
unique      2
top        AD
freq      341
Name: Diagnóstico, dtype: object

In [16]:
fleni600.groupby("Diagnóstico").describe()

Unnamed: 0                                                     \
                 count        mean         std   min    25%    50%    75%   
Diagnóstico                                                                 
AD               341.0  493.800587  290.436408   1.0  229.0  526.0  720.0   
non-AD           297.0  534.670034  300.342314  13.0  289.0  523.0  788.0   

                    StudyDate                                         \
                max     count          mean          std         min   
Diagnóstico                                                            
AD           1042.0     341.0  2.018425e+07  22357.84306  20141104.0   
non-AD       1485.0     297.0  2.018099e+07  22260.91047  20150127.0   

                                                             
                    25%         50%         75%         max  
Diagnóstico                                                  
AD           20170202.0  20180628.0  20201026.0  20221216.0  
non-AD       20160728.0  20180705.0  20200120.0  20230105.0

In [17]:
fleni600.groupby(["PatientSex", "Diagnóstico"]).describe()

Unnamed: 0                                        \
                            count        mean         std   min     25%   
PatientSex Diagnóstico                                                    
F          AD               188.0  508.250000  286.190816   2.0  273.75   
           non-AD           128.0  565.164062  321.103579  14.0  283.75   
M          AD               153.0  476.045752  295.541126   1.0  214.00   
           non-AD           169.0  511.573964  282.389900  13.0  296.00   

                                              StudyDate                \
                          50%     75%     max     count          mean   
PatientSex Diagnóstico                                                  
F          AD           536.0  721.50  1042.0     188.0  2.018296e+07   
           non-AD       602.5  904.75  1024.0     128.0  2.017870e+07   
M          AD           524.0  716.00  1040.0     153.0  2.018585e+07   
           non-AD       490.0  739.00  1485.0     169.0  2.018273e+07   

                                                                           \
                                 std         min          25%         50%   
PatientSex Diagnóstico                                                      
F          AD           21932.984028  20141104.0  20170179.75  20180559.0   
           non-AD       24210.907098  20150127.0  20151110.50  20171108.5   
M          AD           22840.073974  20141104.0  20170216.00  20180726.0   
           non-AD       20565.922453  20150129.0  20161110.00  20181102.0   

                                                 
                                75%         max  
PatientSex Diagnóstico                           
F          AD           20200217.25  20221216.0  
           non-AD       20200149.75  20230105.0  
M          AD           20201209.00  20221216.0  
           non-AD       20191223.00  20221229.0

In [19]:
fleni600.groupby("PatientName").describe()

Unnamed: 0                                      \
                               count    mean std     min     25%     50%   
PatientName                                                                
ABALO^CAMILA DOLORES             1.0  1055.0 NaN  1055.0  1055.0  1055.0   
ABD^MIGUEL ANGEL                 1.0  1006.0 NaN  1006.0  1006.0  1006.0   
ABLIN^ABEL SAUL                  1.0   321.0 NaN   321.0   321.0   321.0   
ABRAHAM^BERTA GUILLERMINA        1.0    62.0 NaN    62.0    62.0    62.0   
ABRAHAM^LILIA MABEL              1.0   212.0 NaN   212.0   212.0   212.0   
...                              ...     ...  ..     ...     ...     ...   
ZUBILLAGA^MARIA AMELIA           1.0   425.0 NaN   425.0   425.0   425.0   
ZUCATTI^STELLE AURORA            1.0  1003.0 NaN  1003.0  1003.0  1003.0   
ZUNINO^MARIA INES                1.0    21.0 NaN    21.0    21.0    21.0   
ZURETTI^MIGUEL ANGEL             1.0   608.0 NaN   608.0   608.0   608.0   
ZYTLER ELBA                      1.0  1094.0 NaN  1094.0  1094.0  1094.0   

                                          StudyDate                  \
                              75%     max     count        mean std   
PatientName                                                           
ABALO^CAMILA DOLORES       1055.0  1055.0       1.0  20141016.0 NaN   
ABD^MIGUEL ANGEL           1006.0  1006.0       1.0  20150416.0 NaN   
ABLIN^ABEL SAUL             321.0   321.0       1.0  20191025.0 NaN   
ABRAHAM^BERTA GUILLERMINA    62.0    62.0       1.0  20220610.0 NaN   
ABRAHAM^LILIA MABEL         212.0   212.0       1.0  20201201.0 NaN   
...                           ...     ...       ...         ...  ..   
ZUBILLAGA^MARIA AMELIA      425.0   425.0       1.0  20190426.0 NaN   
ZUCATTI^STELLE AURORA      1003.0  1003.0       1.0  20150421.0 NaN   
ZUNINO^MARIA INES            21.0    21.0       1.0  20221031.0 NaN   
ZURETTI^MIGUEL ANGEL        608.0   608.0       1.0  20171201.0 NaN   
ZYTLER ELBA                1094.0  1094.0       1.0  20140529.0 NaN   

                                                                           \
                                  min         25%         50%         75%   
PatientName                                                                 
ABALO^CAMILA DOLORES       20141016.0  20141016.0  20141016.0  20141016.0   
ABD^MIGUEL ANGEL           20150416.0  20150416.0  20150416.0  20150416.0   
ABLIN^ABEL SAUL            20191025.0  20191025.0  20191025.0  20191025.0   
ABRAHAM^BERTA GUILLERMINA  20220610.0  20220610.0  20220610.0  20220610.0   
ABRAHAM^LILIA MABEL        20201201.0  20201201.0  20201201.0  20201201.0   
...                               ...         ...         ...         ...   
ZUBILLAGA^MARIA AMELIA     20190426.0  20190426.0  20190426.0  20190426.0   
ZUCATTI^STELLE AURORA      20150421.0  20150421.0  20150421.0  20150421.0   
ZUNINO^MARIA INES          20221031.0  20221031.0  20221031.0  20221031.0   
ZURETTI^MIGUEL ANGEL       20171201.0  20171201.0  20171201.0  20171201.0   
ZYTLER ELBA                20140529.0  20140529.0  20140529.0  20140529.0   

                                       
                                  max  
PatientName                            
ABALO^CAMILA DOLORES       20141016.0  
ABD^MIGUEL ANGEL           20150416.0  
ABLIN^ABEL SAUL            20191025.0  
ABRAHAM^BERTA GUILLERMINA  20220610.0  
ABRAHAM^LILIA MABEL        20201201.0  
...                               ...  
ZUBILLAGA^MARIA AMELIA     20190426.0  
ZUCATTI^STELLE AURORA      20150421.0  
ZUNINO^MARIA INES          20221031.0  
ZURETTI^MIGUEL ANGEL       20171201.0  
ZYTLER ELBA                20140529.0  

[1417 rows x 16 columns]